<a href="https://colab.research.google.com/github/JialeiGuo1108/CASA0025/blob/main/notebooks/w03_questions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 加载数据

In [1]:
%pip install duckdb leafmap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 12.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 12.9 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
import duckdb
#leafmap-Python library to visualize spatial data
import leafmap

In [3]:
print(duckdb.__version__)  # 检查当前 DuckDB 版本

1.0.0


In [4]:
url = "https://open.gishub.org/data/duckdb/nyc_data.db.zip"
leafmap.download_file(url, unzip=True)

Downloading...
From: https://open.gishub.org/data/duckdb/nyc_data.db.zip
To: /home/jovyan/work/CASA0025/notebooks/nyc_data.db.zip
100%|██████████| 8.60M/8.60M [00:00<00:00, 11.9MB/s]

Extracting files...


'/home/jovyan/work/CASA0025/notebooks/nyc_data.db.zip'

In [5]:
con = duckdb.connect('nyc_data.db')
#加载spatial拓展
con.install_extension('spatial')
con.load_extension('spatial')

In [6]:
con.sql("SHOW TABLES;")

┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ nyc_census_blocks   │
│ nyc_homicides       │
│ nyc_neighborhoods   │
│ nyc_streets         │
│ nyc_subway_stations │
└─────────────────────┘

In [7]:
con.sql("""
SELECT name, ST_AsText(geom)
FROM nyc_subway_stations;
""")

┌───────────────────┬──────────────────────────────────────────────┐
│       NAME        │               st_astext(geom)                │
│      varchar      │                   varchar                    │
├───────────────────┼──────────────────────────────────────────────┤
│ Cortlandt St      │ POINT (583521.854408956 4507077.862599085)   │
│ Rector St         │ POINT (583324.4866324601 4506805.373160211)  │
│ South Ferry       │ POINT (583304.1823994748 4506069.654048115)  │
│ 138th St          │ POINT (590250.10594797 4518558.019924332)    │
│ 149th St          │ POINT (590454.7399891173 4519145.719617855)  │
│ 149th St          │ POINT (590465.8934191109 4519168.697483203)  │
│ 161st St          │ POINT (590573.169495527 4520214.766177284)   │
│ 167th St          │ POINT (591252.8314104103 4520950.353355553)  │
│ 167th St          │ POINT (590946.3972262995 4521077.318976877)  │
│ 170th St          │ POINT (591583.6111452815 4521434.846626811)  │
│    ·              │             

### 4.1 Exercises
- What is the area of the 'West Village' neighborhood?

(Hint: The area is given in square meters. **To get an area in hectares, divide by 10000. To get an area in acres, divide by 4047.**)

In [8]:
con.sql("""
SELECT name,ST_Area(geom)/4047, ST_Area(geom)/10000
FROM nyc_neighborhoods
WHERE name='West Village';
""")

┌──────────────┬────────────────────────┬─────────────────────────┐
│     NAME     │ (st_area(geom) / 4047) │ (st_area(geom) / 10000) │
│   varchar    │         double         │         double          │
├──────────────┼────────────────────────┼─────────────────────────┤
│ West Village │      258.1207140223859 │      104.46145296485959 │
└──────────────┴────────────────────────┴─────────────────────────┘

- What is the geometry type of ‘Pelham St’佩勒姆街? The length?

In [14]:
con.sql("""
SELECT name, ST_GeometryType(geom), ST_Length(geom)
FROM nyc_streets
WHERE name = 'Pelham St';
""")

┌───────────┬───────────────────────┬───────────────────┐
│   NAME    │ st_geometrytype(geom) │  st_length(geom)  │
│  varchar  │     geometry_type     │      double       │
├───────────┼───────────────────────┼───────────────────┤
│ Pelham St │ MULTILINESTRING       │ 50.32314951660229 │
└───────────┴───────────────────────┴───────────────────┘

- What is the GeoJSON representation of the 'Broad St' subway station?

In [38]:
con.sql("""
SELECT name, ST_AsGeoJSON(geom)
FROM nyc_subway_stations
WHERE name = 'Broad St';
""")

┌──────────┬──────────────────────────────────────────────────────────────────────┐
│   NAME   │                          st_asgeojson(geom)                          │
│ varchar  │                                 json                                 │
├──────────┼──────────────────────────────────────────────────────────────────────┤
│ Broad St │ {"type":"Point","coordinates":[583571.9059213118,4506714.341192182]} │
└──────────┴──────────────────────────────────────────────────────────────────────┘

- What is the total length of streets (in kilometers) in New York City?

(Hint: The units of measurement of the spatial data are meters, there are 1000 meters in a kilometer.)

In [ ]:
con.sql("""
SELECT sum(ST_length(geom))/1000
FROM nyc_streets;
""")

┌───────────────────────────────┐
│ (sum(st_length(geom)) / 1000) │
│            double             │
├───────────────────────────────┤
│            10418.904717199997 │
└───────────────────────────────┘

- What is the area of Manhattan in acres? (Hint: <font color=#6495ED>both `nyc_census_blocks` and `nyc_neighborhoods` have a **boroname** in them.)

In [22]:
con.sql("""
SELECT ANY_VALUE(BORONAME), sum(ST_Area(geom))/4047
FROM nyc_census_blocks
WHERE BORONAME='Manhattan';
""")

┌─────────────────────┬─────────────────────────────┐
│ any_value(BORONAME) │ (sum(st_area(geom)) / 4047) │
│       varchar       │           double            │
├─────────────────────┼─────────────────────────────┤
│ Manhattan           │          14601.398721554795 │
└─────────────────────┴─────────────────────────────┘

In [21]:
con.sql("""
SELECT ANY_VALUE(BORONAME), sum(ST_Area(geom))/4047
FROM nyc_neighborhoods
WHERE BORONAME='Manhattan';
""")

┌─────────────────────┬─────────────────────────────┐
│ any_value(BORONAME) │ (sum(st_area(geom)) / 4047) │
│       varchar       │           double            │
├─────────────────────┼─────────────────────────────┤
│ Manhattan           │           13965.32012239119 │
└─────────────────────┴─────────────────────────────┘

- What is the most westerly subway station?

In [67]:
con.sql("""
SELECT name, ST_X(geom)
FROM nyc_subway_stations
WHERE ST_X(geom) = (SELECT MIN(ST_X(geom)) FROM nyc_subway_stations);
""")

┌─────────────┬───────────────────┐
│    NAME     │    st_x(geom)     │
│   varchar   │      double       │
├─────────────┼───────────────────┤
│ Tottenville │ 563292.1172580556 │
└─────────────┴───────────────────┘

In [65]:
con.sql("""
SELECT name, ST_X(geom)
FROM nyc_subway_stations
ORDER BY ST_X(geom)
LIMIT 1;
""")

┌─────────────┬───────────────────┐
│    NAME     │    st_x(geom)     │
│   varchar   │      double       │
├─────────────┼───────────────────┤
│ Tottenville │ 563292.1172580556 │
└─────────────┴───────────────────┘

- How long is 'Columbus Cir' (aka Columbus Circle)?

In [31]:
con.sql("""
SELECT name, ST_Length(geom)
FROM nyc_streets
WHERE name='Columbus Cir';
""")

┌──────────────┬───────────────────┐
│     NAME     │  st_length(geom)  │
│   varchar    │      double       │
├──────────────┼───────────────────┤
│ Columbus Cir │ 308.3419936909855 │
└──────────────┴───────────────────┘

- What is the length of streets in New York City, summarized by type?

In [63]:
con.sql("""
SELECT type, Sum(ST_Length(geom)) AS length
FROM nyc_streets
GROUP BY type
ORDER BY length DESC;
""")

┌──────────────────────────────────────────────────┬────────────────────┐
│                       TYPE                       │       length       │
│                     varchar                      │       double       │
├──────────────────────────────────────────────────┼────────────────────┤
│ residential                                      │  8629870.337866059 │
│ motorway                                         │  403622.4781263628 │
│ tertiary                                         │  360394.8790513027 │
│ motorway_link                                    │   294261.419479668 │
│ secondary                                        │  276264.3038979258 │
│ unclassified                                     │  166936.3716044583 │
│ primary                                          │  135034.2330179469 │
│ footway                                          │  71798.48783780965 │
│ service                                          │ 28337.635038596007 │
│ trunk                               

### 5.1 Exercises
- What is the geometry value for the street named 'Atlantic Commons'?

In [54]:
con.sql("""
SELECT name, ST_AsText(geom)
FROM nyc_streets
WHERE name='Atlantic Commons';
""")

┌──────────────────┬──────────────────────────────────────────────────────────────────────────────────────────────┐
│       NAME       │                                       st_astext(geom)                                        │
│     varchar      │                                           varchar                                            │
├──────────────────┼──────────────────────────────────────────────────────────────────────────────────────────────┤
│ Atlantic Commons │ MULTILINESTRING ((586781.7015777241 4504202.153143394, 586863.5196448397 4504215.988170098)) │
└──────────────────┴──────────────────────────────────────────────────────────────────────────────────────────────┘

- What neighborhood and borough is Atlantic Commons in?

In [44]:
#查看neighborhood数据格式
con.sql("FROM nyc_neighborhoods LIMIT 5")

┌───────────┬──────────────────────┬───────────────────────────────────────────────────────────────────────────────────┐
│ BORONAME  │         NAME         │                                       geom                                        │
│  varchar  │       varchar        │                                     geometry                                      │
├───────────┼──────────────────────┼───────────────────────────────────────────────────────────────────────────────────┤
│ Brooklyn  │ Bensonhurst          │ MULTIPOLYGON (((582771.4257198056 4495167.427365481, 584651.2943549604 4497541.…  │
│ Manhattan │ East Village         │ MULTIPOLYGON (((585508.7534890148 4509691.267208001, 586826.3570590394 4508984.…  │
│ Manhattan │ West Village         │ MULTIPOLYGON (((583263.2776595836 4509242.626023987, 583276.8199068634 4509378.…  │
│ The Bronx │ Throggs Neck         │ MULTIPOLYGON (((597640.0090688139 4520272.719938631, 597647.7457808304 4520617.…  │
│ The Bronx │ Wakefield-Williams

In [55]:
#注意括号
con.sql("""
SELECT BORONAME, NAME
FROM nyc_neighborhoods
WHERE ST_Intersects(
geom, 
ST_GeomFromText('MULTILINESTRING ((586781.7015777241 4504202.153143394, 586863.5196448397 4504215.988170098))')
);
""")

┌──────────┬────────────┐
│ BORONAME │    NAME    │
│ varchar  │  varchar   │
├──────────┼────────────┤
│ Brooklyn │ Fort Green │
└──────────┴────────────┘

- What streets does Atlantic Commons join with?

In [56]:
con.sql("""
SELECT name
FROM nyc_streets
WHERE ST_Touches(
geom, 
ST_GeomFromText('MULTILINESTRING ((586781.7015777241 4504202.153143394, 586863.5196448397 4504215.988170098))')
);
""")

┌───────────────┐
│     NAME      │
│    varchar    │
├───────────────┤
│ S Oxford St   │
│ Cumberland St │
└───────────────┘

- Approximately how many people live on (within 50 meters of) Atlantic Commons?

In [58]:
con.sql("""
SELECT sum(popn_total)
FROM nyc_census_blocks
WHERE ST_DWithin(
geom, 
ST_GeomFromText('MULTILINESTRING ((586781.7015777241 4504202.153143394, 586863.5196448397 4504215.988170098))'),
50
);
""")

┌─────────────────┐
│ sum(popn_total) │
│     int128      │
├─────────────────┤
│            1438 │
└─────────────────┘